In [1]:
import pandas as pd
import numpy as np
import datetime as dt

In [2]:
ls data/sports_scores/

astros_1A10.csv*   Rice_10H80.csv*    Tex_15E40.csv*
dynamo_10H30.csv*  rockets_1A10.csv*  UH_10H50.csv*


In [3]:
ls data/crime_beat/

crime_10H30_10_17.csv*  crime_10H80_10_17.csv*  crime_1A10_10_17.csv*
crime_10H50_10_17.csv*  crime_15E40_10_17.csv*


In [4]:
games = 'data/sports_scores/UH_10H50.csv'

crime = 'data/crime_beat/crime_10H50_10_17.csv'

# scores df

In [6]:
df = pd.read_csv(games,index_col='Date').sort_index(ascending=True)

## win column
df['win'] =  np.where(df['Pts'] > df['Opp'],1,0)

# only oposing name and win boolean
df = df[['win','beat']]

# show

df.head()

,win,beat
Date,,
2010-09-04,1,10H50
2010-09-10,1,10H50
2010-09-25,1,10H50
2010-10-09,0,10H50
2010-11-05,0,10H50


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 49 entries, 2010-09-04 to 2017-11-24
Data columns (total 2 columns):
win     49 non-null int64
beat    49 non-null object
dtypes: int64(1), object(1)
memory usage: 1.1+ KB


# Crime df

In [8]:
crime_df = pd.read_csv(crime).sort_index(ascending=True)

crime_df['Date_Hour'] = pd.to_datetime(crime_df['Date_Hour'])
crime_df['Date_Hour'] = crime_df['Date_Hour'].dt.date

crime_df = crime_df.set_index('Date_Hour')


# selectd columns
crime_df = crime_df[['OffenseType','Hour']]

# # count Offense type given date
# date_crimes = crime_df.groupby(crime_df.index)['OffenseType',].count()

# ## show
# date_crimes.head()

In [9]:
crime_df = pd.get_dummies(crime_df)

In [10]:
crime_df.head()

,Hour,OffenseType_Aggravated Assault,OffenseType_Auto Theft,OffenseType_Burglary,OffenseType_Murder,OffenseType_Rape,OffenseType_Robbery,OffenseType_Theft
Date_Hour,,,,,,,,
2010-01-01,9,0,0,0,0,0,0,1
2010-01-01,12,0,0,1,0,0,0,0
2010-01-01,12,0,0,0,0,0,0,1
2010-01-01,18,1,0,0,0,0,0,0
2010-01-01,21,1,0,0,0,0,0,0


# Days Df
- create a df with everyday timestamp from Jan 2010 to Dec 2017

In [11]:
days = pd.date_range(start='01/01/2010', end='12/31/2017')
days = pd.DataFrame(days)

days.columns = ['days']
days = days.set_index('days').sort_index(ascending=True)
days.head()

""
days
2010-01-01
2010-01-02
2010-01-03
2010-01-04
2010-01-05


In [ ]:
365 * 8

In [ ]:
len(days)/365

In [ ]:
days.tail()

# Merge days df with crimes_df

In [12]:
#calendar_crimes = pd.merge(days,date_crimes,  left_index=True, right_index=True, how='left')
calendar_crimes = pd.merge(days,crime_df,  left_index=True, right_index=True, how='left')

calendar_crimes.head()

,Hour,OffenseType_Aggravated Assault,OffenseType_Auto Theft,OffenseType_Burglary,OffenseType_Murder,OffenseType_Rape,OffenseType_Robbery,OffenseType_Theft
2010-01-01,9.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2010-01-01,12.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2010-01-01,12.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2010-01-01,18.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2010-01-01,21.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
calendar_crimes.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8225 entries, 2010-01-01 to 2017-12-31
Data columns (total 8 columns):
Hour                              8017 non-null float64
OffenseType_Aggravated Assault    8017 non-null float64
OffenseType_Auto Theft            8017 non-null float64
OffenseType_Burglary              8017 non-null float64
OffenseType_Murder                8017 non-null float64
OffenseType_Rape                  8017 non-null float64
OffenseType_Robbery               8017 non-null float64
OffenseType_Theft                 8017 non-null float64
dtypes: float64(8)
memory usage: 578.3 KB


In [ ]:
calendar_crimes.OffenseType.value_counts(dropna=False)

## Change nans with 0

In [ ]:
calendar_crimes.OffenseType.replace( np.nan,0, inplace=True)

In [ ]:
calendar_crimes.OffenseType.value_counts(dropna=False)

# merge calendar_crimes with score

In [14]:
merge_data = pd.merge(calendar_crimes,df,  left_index=True, right_index=True, how='left')

# change column names
#merge_data.columns = ['offenses','away_team','win']

merge_data.head()

,Hour,OffenseType_Aggravated Assault,OffenseType_Auto Theft,OffenseType_Burglary,OffenseType_Murder,OffenseType_Rape,OffenseType_Robbery,OffenseType_Theft,win,beat
2010-01-01,9.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,NaN
2010-01-01,12.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,NaN,NaN
2010-01-01,12.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,NaN
2010-01-01,18.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
2010-01-01,21.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN


## Change nan values on beat col

In [ ]:
merge_data.beat.replace( np.nan,'10H50', inplace=True)

In [ ]:
merge_data.beat.value_counts(dropna=False)

In [18]:
len(merge_data[merge_data.Hour.isnull()])

208

## Cleanup colums

In [ ]:
merge_data.columns  = ['crime', 'away_team', 'won','beat']

In [ ]:
merge_data.info()

In [ ]:
merge_data.crime.value_counts(dropna=False)

In [ ]:
merge_data.crime = merge_data.crime.astype(int)

In [ ]:
merge_data.crime.value_counts(dropna=False)

In [ ]:
merge_data

# Display non-nans

In [ ]:
merge_data.dropna(subset=['away_team', 'won'])

## Ad sport col

In [ ]:
merge_data['sport'] = 'cfb'

In [19]:
def game_feature(df):
    if df.win == 1:
        val = 'Won Game'
    elif df.win == 0:
        val = 'Lost Game'
    else:
        val = 'No Game'
    return val

merge_data['game'] = merge_data.apply(game_feature,axis=1)

In [20]:
merge_data.head()

,Hour,OffenseType_Aggravated Assault,OffenseType_Auto Theft,OffenseType_Burglary,OffenseType_Murder,OffenseType_Rape,OffenseType_Robbery,OffenseType_Theft,win,beat,game
2010-01-01,9.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,NaN,No Game
2010-01-01,12.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,NaN,NaN,No Game
2010-01-01,12.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,NaN,No Game
2010-01-01,18.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,No Game
2010-01-01,21.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,No Game


In [21]:
merge_data.to_csv('data/final/DUMMYUH1017_final.csv')

In [22]:
#merge_data.to_csv('data/final/UH1017_final.csv')

In [23]:
merge_data.head()

,Hour,OffenseType_Aggravated Assault,OffenseType_Auto Theft,OffenseType_Burglary,OffenseType_Murder,OffenseType_Rape,OffenseType_Robbery,OffenseType_Theft,win,beat,game
2010-01-01,9.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,NaN,No Game
2010-01-01,12.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,NaN,NaN,No Game
2010-01-01,12.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,NaN,No Game
2010-01-01,18.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,No Game
2010-01-01,21.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,No Game
